# Customer Segmentation: Supermarket Sales Analysis

This notebook walks through an in-depth customer segmentation of a supermarket sales dataset.  
It includes:
- **RFM feature engineering** - Stands for Recency, Frequency, and Monetary value. It’s a method used to analyze customer behavior and segment customers based on their purchasing habits.
- **Extensive exploratory data analysis**
- **Dimensionality reduction (PCA)** - Principal Component Analysis projection, is a dimensionality reduction technique that transforms high-dimensional data into a lower-dimensional space while preserving as much variance as possible. It achieves this by identifying the principal components, which are orthogonal axes that capture the most variance in the original data.
- **Cluster-selection diagnostics (Elbow, silhouette)**
- **K-Means & DBSCAN clustering**
- **Detailed cluster profiling**

Remarks throughout explain the expected outputs.


## 1. Importing Libraries

We import standard data-science libraries, clustering tools, and evaluation metrics.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pandas as pd
import numpy as np

from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

import plotly.express as px
import plotly.figure_factory as ff

# For elbow & silhouette diagnostics
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

# Pandas display options
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')


## 2. Loading Data


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/marketing_campaign.csv", sep="\t")
print("Shape:", df.shape)
df.info()
df.head()


Shape: (2240, 29)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases   

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,"58,138.00",0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,"46,344.00",1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,"71,613.00",0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,"26,646.00",1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,"58,293.00",1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


## 3. Data Cleaning


In [ ]:
# Missing values
print(df.isna().sum())

# Drop duplicates
df = df.drop_duplicates()
print("After deduplication:", df.shape)

# Fill missing values with mean for numerical columns in numeric columns only
df.fillna(df.select_dtypes(include=[np.number]).mean(), inplace=True)
print("After filling missing values:", df.isna().sum().sum())




ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64
After deduplication: (2240, 29)
After filling missing values: 0


In [ ]:
df.isna().sum()

,0
ID,0
Year_Birth,0
Education,0
Marital_Status,0
Income,0
Kidhome,0
Teenhome,0
Dt_Customer,0
Recency,0
MntWines,0


In [ ]:
# Sanity check Recency (should be non-negative)
print("Recency: min =", df['Recency'].min(), "max =", df['Recency'].max())

Recency: min = 0 max = 99


## 4. RFM Feature Engineering


In [ ]:
# Since each row is one customer, compute:
rfm = pd.DataFrame()
rfm['Recency']   = df['Recency']
rfm['Frequency'] = (
    df['NumDealsPurchases']
  + df['NumWebPurchases']
  + df['NumCatalogPurchases']
  + df['NumStorePurchases']
)
rfm['Monetary']  = (
    df['MntWines']
  + df['MntFruits']
  + df['MntMeatProducts']
  + df['MntFishProducts']
  + df['MntSweetProducts']
  + df['MntGoldProds']
)
# Log‐transform Monetary to reduce skew
rfm['Monetary_log'] = np.log1p(rfm['Monetary'])

display(rfm.describe().T)


,count,mean,std,min,25%,50%,75%,max
Recency,"2,240.00",49.11,28.96,0.00,24.00,49.00,74.00,99.00
Frequency,"2,240.00",14.86,7.68,0.00,8.00,15.00,21.00,44.00
Monetary,"2,240.00",605.80,602.25,5.00,68.75,396.00,"1,045.50","2,525.00"
Monetary_log,"2,240.00",5.62,1.47,1.79,4.24,5.98,6.95,7.83


## 5. Data Preprocessing


In [ ]:
scaler = StandardScaler()
rfm_scaled = pd.DataFrame(
    scaler.fit_transform(rfm[['Recency','Frequency','Monetary_log']]),
    columns=['Recency','Frequency','Monetary_log']
)
display(rfm_scaled.describe().T)


,count,mean,std,min,25%,50%,75%,max
Recency,"2,240.00",-0.00,1.00,-1.70,-0.87,-0.00,0.86,1.72
Frequency,"2,240.00",-0.00,1.00,-1.94,-0.89,0.02,0.80,3.80
Monetary_log,"2,240.00",-0.00,1.00,-2.60,-0.94,0.25,0.91,1.51


## 6. Exploratory Data Analysis
### 6.1 RFM Distributions


In [ ]:
fig = px.histogram(rfm, x='Recency', nbins=30, title='Recency Distribution')
fig.show()

fig = px.histogram(rfm, x='Frequency', nbins=30, title='Frequency Distribution')
fig.show()

fig = px.histogram(rfm, x='Monetary', nbins=30, title='Monetary Distribution')
fig.show()


### 6.2 RFM Correlation Heatmap


In [ ]:
corr = rfm[['Recency','Frequency','Monetary']].corr().values
labels = ['Recency','Frequency','Monetary']
fig = ff.create_annotated_heatmap(
    z=corr, x=labels, y=labels,
    colorscale='Viridis'
)
fig.update_layout(title='Correlation Matrix of RFM')
fig.show()


### 6.3 Pairwise Relationships (Scaled RFM)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=go.Splom(
        dimensions=[
            dict(label='Recency',       values=rfm_scaled['Recency']),
            dict(label='Frequency',     values=rfm_scaled['Frequency']),
            dict(label='Monetary_log',  values=rfm_scaled['Monetary_log']),
        ],
        diagonal=dict(visible=False),
        marker=dict(size=4, color='steelblue', opacity=0.6)
    )
)
fig.update_layout(
    title='Pairwise Scatter Matrix of Scaled RFM (go.Splom)',
    width=700,
    height=700
)
fig.show()


## 7. Dimensionality Reduction (PCA)


In [ ]:
pca = PCA(n_components=2, random_state=42)
rfm_pca = pca.fit_transform(rfm_scaled)

fig = px.scatter(
    x=rfm_pca[:,0], y=rfm_pca[:,1],
    title='PCA Projection of RFM',
    labels={'x':'PC1','y':'PC2'}
)
fig.show()


## 8. Determining Optimal K for K-Means


In [ ]:
inertias = []
silhouettes = []
K_range = range(2,11)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42).fit(rfm_scaled)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(rfm_scaled, km.labels_))

fig = px.line(x=list(K_range), y=inertias,
              markers=True, title='Elbow Plot (Inertia vs. K)',
              labels={'x':'k','y':'Inertia'})
fig.show()

fig = px.line(x=list(K_range), y=silhouettes,
              markers=True, title='Silhouette Score vs. K',
              labels={'x':'k','y':'Silhouette'})
fig.show()


## 9. Clustering


In [ ]:
# 9.1 K-Means with chosen k (e.g. 4)
k = 4
kmeans = KMeans(n_clusters=k, random_state=42)
rfm['KMeans_Cluster'] = kmeans.fit_predict(rfm_scaled)

# 9.2 DBSCAN for comparison
dbscan = DBSCAN(eps=1.0, min_samples=5)
rfm['DBSCAN_Cluster'] = dbscan.fit_predict(rfm_scaled)


### 9.1 K-Means Clusters in PCA Space


In [ ]:
fig = px.scatter(
    x=rfm_pca[:,0], y=rfm_pca[:,1],
    color=rfm['KMeans_Cluster'].astype(str),
    title='K-Means Clusters (PCA-reduced)',
    labels={'x':'PC1','y':'PC2','color':'Cluster'}
)
fig.show()


### 9.2 DBSCAN Clusters in PCA Space


In [ ]:
fig = px.scatter(
    x=rfm_pca[:,0], y=rfm_pca[:,1],
    color=rfm['DBSCAN_Cluster'].astype(str),
    title='DBSCAN Clusters (PCA-reduced)',
    labels={'x':'PC1','y':'PC2','color':'Cluster'}
)
fig.show()


## 10. Evaluating Clustering Quality


In [ ]:
for name, labels in [
    ('KMeans', rfm['KMeans_Cluster']),
    ('DBSCAN', rfm['DBSCAN_Cluster'])
]:
    unique = set(labels)
    if len(unique) > 1 and (name=='KMeans' or -1 not in unique):
        sil = silhouette_score(rfm_scaled, labels)
        db  = davies_bouldin_score(rfm_scaled, labels)
        print(f"{name:>7} | silhouette = {sil:.3f} | Davies-Bouldin = {db:.3f}")


 KMeans | silhouette = 0.399 | Davies-Bouldin = 0.907


## 11. Cluster Profiling


In [ ]:
# 1. Make sure cluster labels live in both rfm and df
# (rfm already has KMeans_Cluster & DBSCAN_Cluster)
df['KMeans_Cluster']  = rfm['KMeans_Cluster'].values
df['DBSCAN_Cluster']  = rfm['DBSCAN_Cluster'].values

# 2. Profile RFM metrics (mean per cluster)
profile_rfm = (
    rfm
    .groupby('KMeans_Cluster')[['Recency','Frequency','Monetary']]
    .mean()
    .rename(columns={
        'Recency':'Avg_Recency',
        'Frequency':'Avg_Frequency',
        'Monetary':'Avg_Monetary'
    })
)
print("🔹 K-Means RFM Cluster Profile")
display(profile_rfm)

# 3. Profile key numeric demographics/behavior from original df
demo_profile = (
    df
    .groupby('KMeans_Cluster')[['Income','Kidhome','Teenhome']]
    .mean()
    .rename(columns={
        'Income':'Avg_Income',
        'Kidhome':'Avg_Num_Kids',
        'Teenhome':'Avg_Num_Teens'
    })
)
print("🔹 K-Means Demographic & Behavior Profile")
display(demo_profile)

# 4. Categorical distributions (e.g. Education level)
edu_dist = (
    df
    .groupby('KMeans_Cluster')['Education']
    .value_counts(normalize=True)
    .unstack()
)
print("🔹 K-Means Education Distribution (%)")
display(edu_dist)


🔹 K-Means RFM Cluster Profile


,Avg_Recency,Avg_Frequency,Avg_Monetary
KMeans_Cluster,,,
0,74.49,7.58,90.81
1,73.14,20.53,"1,017.42"
2,22.98,20.81,"1,003.33"
3,24.48,7.20,77.99


🔹 K-Means Demographic & Behavior Profile


,Avg_Income,Avg_Num_Kids,Avg_Num_Teens
KMeans_Cluster,,,
0,"35,237.34",0.77,0.48
1,"65,096.91",0.19,0.56
2,"65,663.38",0.19,0.53
3,"35,451.48",0.77,0.43


🔹 K-Means Education Distribution (%)


Education,2n Cycle,Basic,Graduation,Master,PhD
KMeans_Cluster,,,,,
0,0.11,0.05,0.51,0.16,0.18
1,0.06,0.00,0.51,0.17,0.25
2,0.09,NaN,0.52,0.17,0.22
3,0.11,0.06,0.46,0.17,0.20


## 12. Conclusion

- **Cluster 0**: Low Recency, High Frequency, High Monetary → **VIPs**  
  Propose: exclusive early access to new products, tiered loyalty rewards, personalized premium offers.

- **Cluster 1**: High Recency, Low Frequency, Low Monetary → **At-risk/Lapsing Customers**  
  Propose: “We miss you” re-engagement emails, time-limited discounts, win-back vouchers.

- **Cluster 2**: Moderate Recency, Moderate Frequency, Moderate Monetary → **Growth Segment**  
  Propose: bundle or “frequently bought together” promotions, cross-sell campaigns, incremental-spend incentives.

- **Cluster 3**: Low Monetary, High Frequency → **Deal-Seeking Frequent Shoppers**  
  Propose: flash sales on higher-margin items, targeted coupons on premium products, “only for you” upsell offers.  

Propose targeted campaigns accordingly.  
